### Convolution
- Convolution is an element-wise multiplication of two matrices
followed by a sum.
    1. Take two matrices (which both have the same dimensions).
    2. Multiply them, element-by-element (i.e., not the dot product, just a simple multiplication).
    3. Sum the elements together.
 
- Image ~ Big-matrix
- Kernel/Convolutional Matrix  ~ Tiny maxtrix 
    Odd kernel (3x3) => can get center

In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import argparse
import cv2  

- Convolution Implemented:
- Add zero padding (Replicate padding): For ensuring the spacial size of input image are not reduced.
- The kernel will multiply each input pixel by a small fraction and take the sum – this is exactly the definition of the
average.

In [ ]:
# Construct average blurring kernels used to smooth an image
small_blur = np.ones((7, 7), dtype='float') * (1.0 / (7 * 7))
large_blur = np.ones((21, 21), dtype='float') * (1.0 / (21 * 21))
